In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [4]:
def from_x_to_matrix(x):
    n = int((-3 + np.sqrt(9+8*x.size))//2)
    k = x.size - n
    A = np.zeros((n, n))
    b = np.zeros(n)
    
    #Insert first k coefficients into matrix
    end = 0
    for j in range(n):
        start = end
        end = start + n-j
        A[j, j:] = x[start:end]
        A[j+1:,j] = A[j, j+1:]
    b = x[-n:] #Insert last n coefficients into vector
    return A, b

# Calculate h for single point z
def hi(x, zi):
    A, b = from_x_to_matrix(x)
    return zi.dot(A.dot(zi)) + b.dot(zi) - 1

# Calculate residual r for single point z
def r(x, zi, wi):
    return np.maximum(wi * hi(x, zi), 0)

#Calculate residual vector
def R(x, Z, W):
    m, n = Z.shape
    R = np.zeros(m)
    for i in range(R.size):
        R[i] = r(x, Z[i], W[i])
    return R

# Calculate objective function
def f(x, Z, W):
    m, n = Z.shape
    return np.sum(R(x, Z, W)**2)

# Calculate gradient oh h for single point z
def dhi(x, zi):
    n = zi.size
    k = n*(n+1)//2
    dh = np.zeros(k+n)
    end = 0
    for j in range(n):
        start = end
        end = start + n-j
        dh[start] = zi[j]**2
        dh[start+1:end] = 2 * zi[j] * zi[start+1:end]
    dh[-n:] = zi
    return dh

# Calculate gradient of residual r for  single point z
# h is the hi value for the given point
def dri(x, zi, wi, h = None):
    n = zi.size
    dr = np.zeros(x.size)
    if h == None:
        h = hi(x, zi)
    return (h > 0) * dhi(x, zi) * wi

# Calculate jacobian of residual vector R
def jacobi(x, Z, W, h = None):
    m, n = Z.shape
    J = np.zeros((m, x.size))
    for i in range(m):
        J[i] = dri(x, Z[i], W[i], h)
    return J

# Calculate gradient of objective function
def df(x, Z, W, h = None):
    return 2 * (jacobi(x, Z, W, h).T).dot(R(x, Z, W))

In [5]:
x = np.array((1, 0, 0.25, 0, 0))
Z = np.array((0, 0, 0, 1, 0, 2, 0, 3, 0, 4)).reshape(5, 2)
# Z = np.array((0,0, 1, 0, 2, 0, 3, 0, 4, 0)).reshape(5, 2)
# Z = np.array((0, 0, 1, 1, 2, 2, 3, 3, 4, 4)).reshape(5,2)
# Z = np.array((2, 0, 2, 1, 2, 2, 2, 3, 2, 4)).reshape(5, 2)
W = np.array((1, 1, 1, 1, 1)) 
print('Z: \n', Z)
print('W: \n', W)

Z: 
 [[0 0]
 [0 1]
 [0 2]
 [0 3]
 [0 4]]
W: 
 [1 1 1 1 1]


In [6]:
for i in range(5):
    print(dhi(x, Z[i]))
    print(dri(x, Z[i], W[i]))

[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  1.  0.  1.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  4.  0.  2.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  9.  0.  3.]
[ 0.  0.  9.  0.  3.]
[  0.   0.  16.   0.   4.]
[  0.   0.  16.   0.   4.]


In [7]:
print(jacobi(x, Z, W))
print(df(x, Z, W))

[[  0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.]
 [  0.   0.   9.   0.   3.]
 [  0.   0.  16.   0.   4.]]
[   0.     0.   118.5    0.    31.5]


In [8]:
r(x, Z[0], W[0])

0.0

In [9]:
hi(x, Z[0])

-1.0

In [10]:
R(x, Z, W)

array([ 0.  ,  0.  ,  0.  ,  1.25,  3.  ])

In [11]:
f(x, Z, W)

10.5625

In [12]:
df(x, Z, W)

array([   0. ,    0. ,  118.5,    0. ,   31.5])

In [13]:
from_x_to_matrix(x)

(array([[ 1.  ,  0.  ],
        [ 0.  ,  0.25]]), array([ 0.,  0.]))

In [32]:
m, n = Z.shape
k = n*(n+1)//2

x = np.random.randn(n+k)
p = np.random.randn(n+k)
p = p/np.linalg.norm(p)
f0 = f(x, Z, W)
g = df(x, Z, W).dot(p)
if g == 0:
    print("p: \n", p)
    print(df(x, Z, W))
    
else:
    print("g = %e" %g)
    for ep in 10.0**np.arange(2, -9, -1):
        g_app = (f(x+ep*p, Z, W)-f0)/ep
        error = abs(g_app-g)/abs(g)
        print('ep = %e, error = %e, g_app = %e' % (ep,error, g_app))

g = -5.831224e+02
ep = 1.000000e+02, error = 9.811000e-01, g_app = -1.102099e+01
ep = 1.000000e+01, error = 8.110003e-01, g_app = -1.102099e+02
ep = 1.000000e+00, error = 1.325716e-01, g_app = -5.058169e+02
ep = 1.000000e-01, error = 1.325716e-02, g_app = -5.753918e+02
ep = 1.000000e-02, error = 1.325716e-03, g_app = -5.823493e+02
ep = 1.000000e-03, error = 1.325716e-04, g_app = -5.830451e+02
ep = 1.000000e-04, error = 1.325716e-05, g_app = -5.831146e+02
ep = 1.000000e-05, error = 1.325775e-06, g_app = -5.831216e+02
ep = 1.000000e-06, error = 1.326842e-07, g_app = -5.831223e+02
ep = 1.000000e-07, error = 1.336731e-08, g_app = -5.831224e+02
ep = 1.000000e-08, error = 1.336731e-08, g_app = -5.831224e+02
